INITIALIZATION

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType
from pyspark.sql.functions import trim, col
     

In [0]:
#READ BRONZE TABLE
df = spark.table("workspace.bronze.crm_cust_info")

Silver Transformations

In [0]:
#TRIMMING
for field in df.schema.fields:
    if isinstance(field.dataType, StringType):
        df = df.withColumn(field.name, trim(col(field.name)))

In [0]:
#NORMALIZATION
df = (
    df
    .withColumn(
        "cst_marital_status",
        F.when(F.upper(F.col("cst_marital_status")) == "S", "Single")
         .when(F.upper(F.col("cst_marital_status")) == "M", "Married")
         .otherwise("n/a")
    )
    .withColumn(
        "cst_gndr",
        F.when(F.upper(F.col("cst_gndr")) == "F", "Female")
         .when(F.upper(F.col("cst_gndr")) == "M", "Male")
         .otherwise("n/a")
    )
)

In [0]:
#Remove Records with Missing Customer ID
df = df.filter(col("cst_id").isNotNull())

In [0]:
#RENAMING COLUMN
RENAME_MAP = {
    "cst_id": "customer_id",
    "cst_key": "customer_number",
    "cst_firstname": "first_name",
    "cst_lastname": "last_name",
    "cst_marital_status": "marital_status",
    "cst_gndr": "gender",
    "cst_create_date": "created_date"
}
for old_name, new_name in RENAME_MAP.items():
    df = df.withColumnRenamed(old_name, new_name)

In [0]:
#SANITY CHECK OF DATAFRAME
df.limit(10).display()

WRITING SILVER TABLE

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("workspace.silver.crm_customers")

SANITY CHECK OF SILVER TABLE

In [0]:

%sql
SELECT * FROM workspace.silver.crm_customers LIMIT 10